In [7]:
import pandas as pd
import numpy as np

In [87]:
class Node:
  def __init__(self,X,Y,name,district,docks,id):
    self.X = X
    self.Y = Y
    self.name = name
    self.district = district
    self.docks = docks
    self.ID = id
  
  def nodePrint(self):
    print("Name - ",self.name)
    print("(X,Y) - (",self.X,",",self.Y,")")
    print("Docks - ",self.docks)
    print("ID - ",self.ID)


class Graph:
  def __init__(self,num):
    self.numVertices = num
    self.graph = [[0 for i in range(num)] for j in range(num)]      #Create a 2D matrix with all 0s of size num
    self.vertices = []
  
  def NeighboursOf(self,vert):
    neighbours = []
    for i in range(self.numVertices):
      if self.graph[vert][i]!=0:
        neighbours.append(i)
    return neighbours

In [88]:
# Loading the graph.
# Need to run the cells where the two classes are defined first
import pickle
f = open("Graph g.pkl","rb")
G = pickle.load(f)
f.close()
print(len(G.vertices))

245


In [37]:
np.array(G.graph).shape

(245, 245)

In [38]:
type(G.vertices[0])

__main__.Node

In [39]:
len(G.graph)

245

In [89]:
G.vertices[0].nodePrint()

Name -  175 N Harvard St
(X,Y) - ( -7918062.31703125 , 5215631.0608375 )
Docks -  18
ID -  0


In [ ]:
G.NeighboursOf(0)

In [171]:
# References
# https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm
# https://www.geeksforgeeks.org/dijkstras-shortest-path-algorithm-greedy-algo-7/
# https://www.udacity.com/blog/2021/10/implementing-dijkstras-algorithm-in-python.html


def selfDjik(graph,source,target=0,groupSize=1):
  distanceToSrc = [float('inf')]*graph.numVertices
  distanceToSrc[source] = 0
  visited = [False]*graph.numVertices
  prev = {}
  for i in range(graph.numVertices):
    
    temp = float('inf')
    for j in range(graph.numVertices):
      if distanceToSrc[j] < temp and visited[j] == False and graph.vertices[j].docks>=groupSize:
        temp = distanceToSrc[j]
        minIndex = j
    if 'minIndex' not in locals():
      print("Cannot reach with given group size")
      return None,None,None
    visited[minIndex] = True
    for y in range(graph.numVertices):
      if graph.graph[minIndex][y]>0 and visited[y]==False:
        if distanceToSrc[y]>distanceToSrc[minIndex]+graph.graph[minIndex][y]:
          distanceToSrc[y] = distanceToSrc[minIndex]+graph.graph[minIndex][y]
          prev[y] = minIndex
  

  path = []
  vertex = target
  while vertex!=source:
    path.append(vertex)
    vertex = prev[vertex]
  path.append(vertex)
  return distanceToSrc, prev, path


In [179]:
di,pr,path = selfDjik(G,0,6)

In [181]:
# Path with no specified group size ie single person
path

[6, 196, 61, 60, 242, 199, 0]

In [175]:
di, pr, path = selfDjik(G,0,6,17)

In [176]:
# Alternate route
path

[6, 70, 189, 106, 78, 242, 173, 0]

In [178]:
# Too large a group
di, pr, path = selfDjik(G,0,6,20)

Cannot reach with given group size
